# Variational Autoencoders With Alternative Bottlenecks  
Authors:
| Name | Student ID |
|------|------------|
| Olivia Sommer Droob | S214696 |
| Karoline Klan Hansen | S214638 |
| Martha Falkesgaard Nybroe | S214692 |
| Signe Djernis Olsen | S206759 |
| Bella Strandfort | S214205 |

# Table of Contents

1. [Project Overview](#1.-project-overview)
2. [Hydra Configuration System](#2.-hydra-configuration-system)
    - [base_config.yaml](#base_configyaml)
    - [wandb_sweep_config.yaml](#wandb_sweep-configyaml)
3. [Data Pipeline](#3.-data-pipeline)
    - [data.py](#datapy)
    - [Dataset filtering and class selection](#dataset-filtering)
    - [Dataloaders and splitting](#dataloaders-and-splitting)
4. [Model Implementations](#4.-model-implementations)
    - [Gaussian Bottleneck](#gaussian-bottleneck)
    - [Dirichlet Bottleneck](#dirichlet-bottleneck)
    - [Continuous Categorical (CC) Bottleneck](#cc-bottleneck)
5. [Training Pipeline](#5.-training-pipeline)
    - [train.py](#trainpy)
    - [Training loop](#training-loop)
    - [Early stopping](#early-stopping)
    - [Checkpoint saving and loading](#checkpointing)
    - [WandB integration](#wandb-integration)
6. [Evaluation Scripts](#6.-evaluation-scripts)
    - [evaluate.py](#evaluatepy)
    - [evaluate_multiple.py](#evaluate_multiplepy)
    - [Latent space visualization (t-SNE)](#latent-space-visualization)
    - [Reconstruction grids](#reconstruction-grids)
7. [Visualization Utilities](#visualization-utilities)
    - [visualize.py](#visualizepy)
8. [Final Findings](#final-findings)
    - [Gaussian vs. Dirichlet vs. CC](#gauss-vs-dir-vs-cc)
    - [Latent structure comparison](#latent-structure)
    - [Reconstruction comparison](#reconstruction-comparison)
    - [Overall conclusions](#overall-conclusions)

# 1. Project Overview


# 2. Hydra Configuration System
Explain configuration setup

In [ ]:
#insert code

# 3. Data Pipeline

Explain dataloaders...



In [ ]:
#insert code

# 4. Model Implementations
This is the most important section ... explain the models

## Gaussian Bottleneck


In [ ]:
# insert code for gauss

## Dirichlet Bottleneck


In [ ]:
# insert code for dir

## CC Bottleneck


In [ ]:
# insert code for cc

# 5. Training Pipeline


In [ ]:
# insert explanations and code